In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

dataset
statoil-iceberg-classifier-challenge



In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from os.path import join as opj
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pylab
plt.rcParams['figure.figsize'] = 10,10

%matplotlib inline

##### 모르는 내용들
1. Stratified Shuffle Split
- Spllit : 분할. 데이터셋을 훈련과 cv셋으로 나누는 것.
- Shuffle : 샘플링
- Stratified : 타켓 분포에 맞춰서 샘플링

2. from os.path import join as opj
- 파일 경로를 효율적이고 안전하게 다루기 위함
- os : 파이썬이 다양한 운영체제 소프트웨어와 상호작용할 수 있게 해주는 내장 모듈
- os.path : 파일 경로를 다루는 함수들을 포함하고 있음
- join from os.path : 여러 개의 문자열을 지능적으로 결합하여, 하나의 파일 경로로 만들어줌.
- opj -> os.path.join과 같은 긴 이름을 짧게!

3. from mpl_toolkits.mplot3d imprt Axes3D
- mpt_toolkits : matplotlib에서 특수한 그래프 기능을 추가해주는 툴킷 모음
- mplot3d : 3D 산점도, 3D 표면도 등 3D 그래프를 그리기 위한 전용 툴킷
- Axes3D : 3차원 축을 생성하는 핵심 객체.

In [3]:
train = pd.read_json("/kaggle/input/dataset/train.json/data/processed/train.json")
target_train = train['is_iceberg']
test = pd.read_json('/kaggle/input/dataset/test.json/data/processed/test.json')

Keras provoed the implementation of pretrained VGG, in it's library so, we do not have to build the network by ourselves. Here we are removing the last layer of VGG and putting our sigmoid layer for binary predictions.

The following code will NOT WORK, since on kaggle notebook, the weights of model cannot be downloaded. However, you can copy paste the code in your own notebook to make it work.

### json 파일 살펴보기

In [4]:
train.head()

,id,band_1,band_2,inc_angle,is_iceberg
0,dfd5f913,"[-27.878360999999998, -27.15416, -28.668615, -...","[-27.154118, -29.537888, -31.0306, -32.190483,...",43.9239,0
1,e25388fd,"[-12.242375, -14.920304999999999, -14.920363, ...","[-31.506321, -27.984554, -26.645678, -23.76760...",38.1562,0
2,58b2aaa0,"[-24.603676, -24.603714, -24.871029, -23.15277...","[-24.870956, -24.092632, -20.653963, -19.41104...",45.2859,1
3,4cfc3a18,"[-22.454607, -23.082819, -23.998013, -23.99805...","[-27.889421, -27.519794, -27.165262, -29.10350...",43.8306,0
4,271f93f4,"[-26.006956, -23.164886, -23.164886, -26.89116...","[-27.206915, -30.259186, -30.259186, -23.16495...",35.6256,0


### Block 1 데이터 로딩 및 전처리 Data Loading & Preprocessing

In [5]:
# 타겟 변수를 따로 저장
target_train = train['is_iceberg']

# inc_angle 열을 숫자 형태로 변환. 변환할 수 없는 값은 NaA 결측치 처리
test['inc_angle'] = pd.to_numeric(test['inc_angle'], errors = 'coerce')
train['inc_angle'] = pd.to_numeric(test['inc_angle'], errors = 'coerce')

# 결측치를 발 ㅗ이전의 유효한 값으로 채움 (pad 방식)
train['inc_angle'] = train['inc_angle'].fillna(method = 'pad')

# 전처리된 각도 정보를 새로운 변수에 할당
X_angle = train['inc_angle']
X_test_angle = test['inc_angle']

/tmp/ipykernel_19/1143497653.py:9: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  train['inc_angle'] = train['inc_angle'].fillna(method = 'pad')


Concept

1. `pd.to_numeric(..., erros = 'coerce')` : inc_angle(입사각) 열에는 가끔 숫자가 아닌 문자열 na이 섞여있을 수 있는데, 이 열의 모든 값을 숫자 (float)타입으로 바꾸는 역할! `erros = 'coerce'` 옵션은 만약 어떤 값이 숫자로 바뀔 수 없다면, 그 값을 강제로 NaN으로 만들라는 의미.

2. `fillna(method='pad')` : `coerce` 옵션 떄문에 생긴 NaN 값들을 처리하는 것. NaN 결측치를 pad 방식으로 채우는데, pad 방식은, 바로 위에 있는 유효한 값으로 그대로 복사해서 채워넣는 것! 데이터가 특정 순서로 정렬되어있을 때 유용한 방식.

### 블록 2 : 이미지 데이터 준비 및 채널 결합. (Image Data Preparation & Channel Combination)

In [6]:
# 훈련 데이터의 band_1, band_2 이미지 데이터를 75x75 크기의 넘파이 배열로 변환
X_band_1 = np.array([np.array(band).astype(np.float32).reshape(75,75) for band in train['band_1']])
X_band_2 = np.array([np.array(band).astype(np.float32).reshape(75,75) for band in train['band_2']])

# band_1과 band_2의 평균값으로 세번 째 채널 생성
X_band_3 = (X_band_1 + X_band_2) / 2

# 결합 concatenate
X_train = np.concatenate([X_band_1[:,:,:,np.newaxis],
                          X_band_2[:,:,:,np.newaxis],
                          X_band_3[:,:,:,np.newaxis]], axis = -1)

# Test Data Set에 대해서도 위와 동일하 과정을 반복
X_band_test_1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test['band_1']])
X_band_test_2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test['band_2']])
X_band_test_3 = (X_band_test_1 + X_band_test_2) / 2
X_test = np.concatenate([X_band_test_1[:,:,:, np.newaxis],
                         X_band_test_2[:,:,:, np.newaxis],
                         X_band_test_3[:,:,:, np.newaxis]], axis = -1)

### 블록 3 : Keras 라이브러리 임포트

In [7]:
# Keras and related library import

# TensorFlow와 Keras 관련 라이브러리 임포트

# 기본 시각화 도구
from matplotlib import pyplot

# 1. 옵티마이저 (Optimizers)
from tensorflow.keras.optimizers import RMSprop, Adam, SGD

# 2. 모델 아키텍처 (Model Architecture)
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (Conv2D, MaxPooling2D, Dense, Dropout, 
                                     Input, Flatten, Activation, GlobalMaxPooling2D, 
                                     BatchNormalization, Concatenate, concatenate, 
                                     LSTM, LeakyReLU, PReLU)
from tensorflow.keras import initializers

# 3. 훈련 보조 도구 (Training Helpers)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

# 4. 사전 학습된 모델 및 데이터셋 (Pre-trained Models & Datasets)
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image

2025-06-12 06:22:45.050271: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749709365.213667      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749709365.261230      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


##### 라이브러리 역할 및 딥러닝 개념 설명

0. 기본 시각화 도구
- matplotlib

1. 옵티마이저
- `from tensorflow.keras.optimizers import RMSprop, Adam, SGD`
    - 딥러닝 개념: 최적화. 옵티마이저는 가중치의 학습을 어떻게 최적화할 것인지 정하는 알고리즘.
        -  SGD : Stochastic Gradient Descent : 기본 경사하강
        -  RMSprop, Adam : SGD의 단점을 보완하여 더 빠르고 안정적으로 최적의 가중치를 찾도록 개선된 알고리즘. 아담은, 방향과 속도를 가중평균치를 이용하여 빠르게 수렴하도록 해줌!

2. Model Architecture
- `from tensorflow.keras.models import Sequential, Model`
- 이 그룹은 신경망의 구조, 즉 뼈대를 만드는 데 사용되는 부품들.
- 역할 : 케라스에서 모델의 전체 구조를 정의하는 방법은 총 두 가지.
    - 딥러닝 개념 : 신경망 아키텍쳐 설계
        - Sequential : 이름처럼, 레이어를 순차적으로 차곡차곡 쌓아 올리는 간단한 모델을 만들 때 사용
        - Model : Input과 함께 사용되며, 여러 개의 입력과 출력을 갖거나, 중간에 레이어가 갈라지고 합쳐지는 등, 복잡한 비선형 구조의 모델(functional API)을 만들 때 사용.
 
- `from tensorflow.keras.layers import (Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation, GlobalMaxPooling2D, BatchNormalization, Concatenate, concatenate, LSTM, LeakyReLU, PReLU)`
    - 역할 : 신경망을 구성하는 각 층 layer. 각 레이어는 특정 계산을 수행!
    - 딥러닝 개념 :
        - Conv2D, MaxPoolig2D, GlobaldMaxPooling2D : 합성곱 신경망의 핵심 레이어.
            - Conv2D :  이미지에 필터를 적용해, feature map 생성 (각 필터마다, 경계산, 질감 감지라던가)
            - MaxPooling2D : 특징맵의 크기르 줄여, 중요한 특징만 남기고 계산량 감소시킴
         - Input, Flatten, Dense : 완전 연결 신경망 구성
             - Input : 모델의 입력을 정의 (Functional API에서 사용)
             - Flatten : 다차원의 특징맵을 1차원 벡터 평탄화
             - Dense : 기본 신경망 레이어
         - Activation, LeakyReLu, PReLu : 활성화 함수. 레이어에 비선형성을 추가.
         - Dropout : 정규화 기법 중 하나. 매 학습마다, 무작위로 특정 뉴런 비활성화
         - BatchNormalization : 정규화 기법 중 하나. 각 레이어를 통과한 데이터의 분포를 평균 0, 분산 1로 정규화하여, 학습 과정을 안정시키고 속도를 향상.
         - Concatenate, concatenate, LSTM : 조금 더 특수한 레이어들.
             - Concatenate : 여러 레이어의 출력을 하나로 합칠 때 사용(다중 입력 모델)
             - LSTM : 순환 신경망 RNN의 한 종류로, 시계열 데이터나 자영너처럼 순서가 중요한 데이터를 처리하는 데 사용.

3. Training Heleprs 훈련 보조 도구
`from tensorflow.keras.preprocessing.image import ImageDataGenerator`
    - 역할 : 실시간으로 이미지 데이터를 증강(augment)하여 모델에 공급하는 제너레이터를 만듦
    - 딥러닝 개념 : 데이터 증강. 가지고 있는 이미지 데이터에 회전,확대/축소/좌우 반전 등 미세한 변형을 가하여 훈련 데이터의 양을 인위적으로 늘리는 기술. 모델이 다양한 상황에 대처하는 능력(일반화 성능)을 높여주고 과적합을 줄이는 데 큰 도움이 됨.

- `from tensorflow.keras.callbakcs import ModelCheckpoint, callback, EarlyStopping`
    - 역할 : 모델 훈련 과정 중간중간에 특정 작업을 수행하도록 하는 callback 함수들
    - 딥러닝 개념 : 훈련 모니터링 및 제어
        - ModelCheckpoint : 훈련 중 검증 성능이 가장 좋았을 때의 모델 가중치를 자동으로 저장
        - EarlyStopping : 검증 성능이 일정 기간 동안 더 이상 향상되지 않으면, 훈련을 조기 종료시켜 시간 낭비와 과적합을 막음.

4. Pre-trained Models & Datasets
- `from tensorflow.keras.datasets import cifar10`
    - 역할 : cifar10은 클래스 10개로 구성된 60000장의 작은 이미지 데이터셋. Keras는 이처럼 널리 사용되는 벤치마크 데이터셋을 쉽게 불러올 수 있는 기능을 제공
    - 딥러닝 개념 : 벤치마킹(Benchmarking). 표준화된 데이터셋을 사용하면, 내가 만든 모델의 성능을 다른 모델들과 객관적으로 비교할 수 있음.
- `from tensorflow.keras.apllications... import VGG16, InceptionV3, ...`
    - 역할 : ImageNet과 같은 거대한 데이터셋으로 이미 학습이 완료된, 성능이 검증된 유명한 모델 아키텍처들을 불러옴.
    - 딥러닝 개념 : 전이학습 (Transfer Learning) : '바퀴를 다시 발명하지 않는다'는 철학과 같다! 수백만 장의 이미지로 학습된 모델이 가진 지식(특징 추출 능력)을 나의 새로운 (상대적으로 작은)데이터셋 문제에 가져와 활용하는 기법. 밑바닥부터 학습하는 것보다 훨씬 빠르고 높은 성능을 얻을 수 있어 널리 사용됨.
- `from tensorflow.keras.applications.vgg16 import process_input`
    - 역할 : 특정 사전 학습 모델(여기서는 vgg16)이 학습될 때 사용했던 것과 똑같은 방식으로 입력 이미지를 전처리(e.g., 픽셀 값 스케일링, 채널 순서 변경 등)해주는 함수
    - 딥러닝 개념 : 모델별 전처리 일관성. 전이학습을 제대로 수행하려면, 내가 사용할 이미지도 원래 모델이 학습했던 데이터와 동일한 분포 형식으로 만들어주어야 함! 그래야 이 모델이 그 역할을 정확히 수행해줌. 요리사에는 요리사에게 걸맞는 재료를!

## 몰랐던 내용들 정리

1. Adam 옵티마이저 : Momentum과 RMSprop의 결합
Adam OPtimizer는 현재 가장 널리 쓰이는 최적화 알고리즘 중 하나. 그 이유는 경사하강법의 두 가지 주요한 문제점을 해결하는 Momentum과 RMSprop의 장점을 모두 채택했기 때문!

가. Momentum(관성)
- 해결하려는 문제 : 일반적인 경사하강법은 경사의 방향이 계속해서 크게 바뀔 때(지그재그로 움직이므로) 학습이 매우 느려지는 문제가 있습니다.
- 핵심 아이디어 : 모멘텀은 이름처럼 물리적인 관성의 개념을 도입합니다. 현재의 경사 방향 뿐만 아니라, 과거에 이동해왔던 방향을 일정 부분 유지하려는 설징을 추가!
    - 작동 방식 : '지수 가중 이동 평균 Exponentially Weighted Moving Average'를 사용하여, 과거 경사들의 평균적인 방향 계산. 이로 인해, 불필요한 진동이 줄어들고, 직선적으로 수렴하게 만듦.
 
나. RMSprop Root mean Sqaure Propagation
- 해결하려는 문제 : 모든 파라미터(가중치)에 동일한 학습률을 적용하는 것은 비효율적. 어떤 파라미터는 이미 최적값에 가까워 조금만 움직여야 하고, 어떤 파라미터는 아직 멀리 있어 더 많이 움직여야 함.
- 핵심 아이디어 : 각 파라미터마다 적응적인 adaptive 학습률을 적용. 즉, 파라미터의 변화가 많았는지 적었는지에 따라 학습률을 조정.
    - 작동 방식 : 이 역시, 지수 가중 이동 평균을 사용하지만, 경사의 방향이 아닌, 경사 제곱값의 평균을 추적. 이는 경사 크기의 변화량을 나타냄.
    - 효과 :
        - 1. 적응적 학습률 : 최근에 경사가 컸던 파라미터는 학습률을 작게 만들고, 경사가 작았던 파라미터는 학습률을 크게 만듦
          2. 안정적인 학습 : 이를 통해, 변화가 심한 파라미터는 조심스럽게 업데이트하고, 변화가 거의 없던 파라미터는 더 과감하게 업데이트하여 ,전체적으로 안정적ㅈ이고 효율적인 학습이 가능.
     
Adam (Adaptive Moemnt Estimation)
Adam은 이 두 가지를 모두 합친 것. 이동 방향은 Momentum처럼 과거의 경사 방향을 고려하여 결정하고, 학습률의 크기는 RMSprop처럼 과거 경사 크기에 따라 적응적으로 조절.

2. MaxPooling2D vs GlobalMaxPooling 2D
두 레이어 모두 CNN에서 특징 맵의 크기를 줄리는 풀링 연산을 수행하지만, 그 방식과 목적에서 명확한 차이가 존재.

가. Maxpooling2D
- 작동 방식 : pool_size (e.g. 2x2)와 strides (e.g., 2)에 따라, 특징맵 위를 창문 window이 이동하면서, 각 창문 영역에서 가장 큰 값 하나만을 추출.
- 목적 :
    1. 특징 맵의 공가적 차원 축소 : 계산량을 줄이고, 모델의 파라미터 감소
    2. 지역적 특징 강조. 특정 지역에서 가장 활성화가 강한 특징을 뽑아내어, 이미지 내 객체의 위치가 조금 변하거나, 회전하더라도 동일한 특징을 잡아낼 수 있도록 함. Translation Invariance
- 사용 위치 : 주로 CNN 아키텍처의 중간 부분에서 Conv2D 레이어 다음에 반복적으로 사용되어, 점진적으로 이미지의 공간적 크기를 줄이고 특징을 압축.

나. GlobalMaxPooling2D
- 작동 방식 : 창문을 이동시키는 것이 아니라, 각 특징 맵 채널 전체에서 가장 큰 값 하나만을 추출. (8, 8, 128) 크기의 특징맵이 입력되면, 각 128개의 채널에서 가장 큰 값 하나씩을 뽑아 (128, ) 크기의 1차원 벡터를 출력.
- 목적 :
    1. 분류를 위한 최종 벡터화 : CNN의 특징 추출 부분의 최종 출력을 분류기 Classifier에 넣기 전에, 다차원의 특징맵을 1차원 벡터로 변환하는 역할. Flatten 레이어와 유사한 목적!
    2. 파라미터 수 감소 : Flatten은 모든 값을 그대로 펴기 때문에, 파라미터 수가 매우 커질 수 있지만, GlobalMaxPooling2D는 채널당 하나의 값만 남기므로, 파라미터 수를 획기적으로 줄여, 과적합을 방지.
- 사용 위치 : 주로, CNN 아키텍처의 맨 마지막, 즉 특징 추출 부분과 완전 연결 분류기 사이에 위치.

3. 활성화 함수 : Activation, LeakyReLU, PReLU

가. Activation은 Keras에서 활성화 함수를 하나의 독립적인 레이어처럼 사용할 때 씀. `Dense(units = 64, activation = 'relu')`처럼 레이어의 인자로 직접 지정할 수도 있지만, Activation('relu')처럼 분리하면 코드의 유연성이 높아짐.

나. LeakyReLU
- 배경 : 가장 널리 쓰이는 활성화 함수인 ReLU는 입력값이 0보다 작으면 항상 0을 출력. 이로 인해 일부 뉴런이 훈련 과정에서 아예 비활성화되어 다시는 활성화되지 않는 Dying ReLU 문제가 발생.
- 핵심 아이디어 : 입력값이 0보다 작을 때 0을 출력하는 대신, 아주 작은 음수 알파 0.01을 적용.

다. PReLU (Parametric ReLU)
- 배경: LeakyReLU의 아이디어를 한 단계 더 발전시킨 것입니다.

- 핵심 아이디어: LeakyReLU에서 음수 기울기 α를 0.01과 같은 고정된 값으로 사용하는 대신, 이 α 값 자체를 **학습 가능한 파라미터(trainable parameter)**로 만듭니다.

- 수식: LeakyReLU와 동일하지만, α가 역전파(backpropagation)를 통해 데이터로부터 최적의 값을 학습합니다.

- 
- 효과: 데이터에 가장 적합한 음수 기울기를 모델이 스스로 찾게 하므로, LeakyReLU보다 더 유연하고 높은 성능을 기대할 수 있습니다. 다만, 파라미터가 추가되므로 과적합의 위험이 미세하게 증가할 수 있습니다.


4. 특수한 레이어: Concatenate, LSTM


가. Concatenate
- 역할: 두 개 이상의 레이어 출력을 하나의 텐서(tensor)로 합치는 역할을 합니다.
- 작동 방식: 지정된 축(axis)을 기준으로 텐서들을 그대로 이어 붙입니다. 예를 들어, (None, 128) 크기의 텐서 A와 (None, 64) 크기의 텐서 B를 마지막 축(axis=-1) 기준으로 합치면, (None, 192) 크기의 새로운 텐서가 생성됩니다.
- 주요 사용 사례:
    - 다중 입력 모델 (Multi-input Models): 이미지 특징과 메타데이터(e.g., 각도, 나이) 특징을 결합하는 것처럼, 서로 다른 소스에서 나온 특징들을 합쳐서 최종 판단을 내리는 모델을 만들 때 필수적입니다.
    - Inception, ResNet과 같은 복잡한 아키텍처: 모델 내부에서 여러 갈래로 나뉜 경로의 출력을 다시 하나로 합칠 때 사용됩니다.
      
나. LSTM (Long Short-Term Memory)
- 역할: 순서(sequence)가 있는 데이터의 패턴을 학습하기 위해 설계된 **순환 신경망(RNN)**의 한 종류입니다.
- 해결하려는 문제: 기본적인 RNN은 시퀀스가 길어질수록 앞쪽의 정보가 뒤쪽으로 전달되면서 점점 희미해지는 **'장기 의존성 문제(Long-term dependency problem)'**를 가집니다. (e.g., 긴 문장의 맨 앞에 있는 주어를 문장 끝에서 기억하지 못하는 문제)
- 핵심 아이디어: '셀 상태(Cell State)'라는 컨베이어 벨트 같은 정보 흐름을 만들고, **3개의 게이트(Gate)**를 통해 이 흐름을 정교하게 제어합니다.
    - Forget Gate: 과거 정보 중 무엇을 잊어버릴지 결정합니다.
    - Input Gate: 현재 입력 정보 중 무엇을 셀 상태에 저장할지 결정합니다.
    - Output Gate: 셀 상태 정보 중 무엇을 다음 시점으로 출력할지 결정합니다.
        - 주요 사용 사례: 자연어 처리(번역, 챗봇), 시계열 데이터 예측(주가, 날씨), 음성 인식 등 순서와 맥락이 중요한 모든 분야에서 강력한 성능을 발휘합니다.

6. 전이 학습 모델 (Transfer Learning Models)
전이 학습은 대규모 데이터셋(주로 ImageNet)으로 미리 학습된 모델의 지식을 가져와 새로운 문제에 적용하는 강력한 기법입니다.

- VGG16 / VGG19:

    - 특징: 3x3의 작은 컨볼루션 필터를 반복적으로 쌓아 네트워크의 깊이를 깊게 만든 모델입니다. 구조가 매우 단순하고 직관적이어서 이해하고 수정하기 쉽습니다. 16/19는 모델의 깊이(가중치를 가진 레이어 수)를 의미합니다.
    - 강점: 구조의 단순함 덕분에 다양한 문제에 대한 훌륭한 **기본 특징 추출기(baseline feature extractor)**로 널리 사용됩니다.


- InceptionV3:

  - 특징: **'인셉션 모듈(Inception Module)'**이라는 독특한 블록을 사용합니다. 이 모듈은 1x1, 3x3, 5x5 등 다양한 크기의 컨볼루션 필터를 병렬로 적용한 뒤 그 결과를 합칩니다.
    - 강점: 네트워크가 스스로 이미지에서 어떤 크기의 특징을 보는 것이 가장 효과적인지 학습하게 하므로, 적은 파라미터로도 매우 높은 성능을 낼 수 있습니다. 연산 효율성이 뛰어납니다.
      
- Xception (Extreme Inception):

    - 특징: 인셉션의 아이디어를 극단적으로 발전시킨 모델입니다. **'깊이별 분리 합성곱(Depthwise Separable Convolution)'**을 기반으로 합니다. 이는 공간적 특징(가로, 세로)과 채널 간 특징을 분리하여 학습하므로, 파라미터 수와 계산량을 획기적으로 줄입니다.
    - 강점: VGG나 Inception보다 훨씬 적은 파라미터로 더 높은 성능을 달성하는, 매우 효율적인 아키텍처입니다.
      
- MobileNet:

    - 특징: Xception처럼 '깊이별 분리 합성곱'을 핵심으로 사용하며, 이름에서 알 수 있듯이 모바일 및 임베디드 기기와 같이 컴퓨팅 자원이 제한된 환경을 위해 설계되었습니다.
    - 강점: 모델의 크기가 매우 작고 추론 속도가 빠르면서도 준수한 성능을 유지합니다. 실시간 객체 탐지 등 속도가 중요한 애플리케이션에 적합합니다.

##### 블록 4 데이터 증강 다중 입력 제너레이터. Data Augmentation & Generator for Multiple Inputs



In [8]:
# Data Augmentation을 위한 ImageDataGenerator 객체 생성

batch_size = 64
gen = ImageDataGenerator(horizontal_flip = True,
                        vertical_flip = True,
                        zoom_range = 0.2,
                        rotation_range=10)

# 두 개의 입력(이밎, 각도)를 받는 모델을 위한 커스텀 제너레이터 함수
def gen_flow_for_two_inputs(X1, X2, y):
    genX1 = gen.flow(X1, y, batch_size = batch_size, seed = 55)
    genX2 = gen.flow(X1, X2, batch_size = batch_size, seed= 55) # 여기서 X1은 임시, X2가 실제 사용됨
    while True:
        #next는 이제 메소드가 아니라, 함수로 사용
        X1i = next(genX1)
        X2i = next(genX2)
        yield (X1i[0], X2i[1]), X1i[1]

# 모델 훈련 중 사용할 콜백 함수들을 정의하는 함수
def get_callbacks(filepath, patience = 2):
    es = EarlyStopping('val_loss', patience = 10, mode = 'min')
    msave = ModelCheckpoint(filepath, save_best_only = True)
    return [es, msave]

##### 상세 설명:
이 블록은 모델의 일반화 성능을 높이기 위한, 데이터 증강을 설정!

1. `ImageDataGenerator` : Keras에서 제공하는 데이터 증강 도구. 기존 이미지에 좌우/상하 반전, 확대/축소, 회전 등 미세한 변화를 무작위로 적용하여, 마치 새로운 데이터인 것처럼 훈련 데이터를 늘려주는 효과. `ImageDataGenerator`를 통해, 이미지 변경 객체를 생성.

2. `gen_flow_for_two_inputs` 함수 : 이 코드의 학샘 중 하나. 우리가 만들 모델은 두 종류의 입력, 즉 이미지 데이터(X1)과 각도 데이터 (X2)를 동시에 받습니다. 하지만, Keras의 기본 제너레이터는 보통 하나의 입력 X와 하나의 출력 y만 처리합니다. 이 함수는 이 문제를 해결하기 위해, 만들어진 커스텀 제너레이터입니다!
    - seed = 55 : genX1과 genX2에 동일한 seed를 사용하는 것이 매우 중요합니다. 이렇게 해야 genX1에서 첫 번째 이미지를 회전시켰다면, genX2에서도 첫 번째 이미지에 해당하는 각도 데이터가 정확히 같은 쌍으로 묶여서 나옵니다. 즉, 데이터의 짝이 어긋나지 않게 해줍니다.
  
    - yiled [X1i[0], X2i[1]], X1i[1] : 이 부분이 실제로 모델에 데이터를 전달하는 부분. 한 번에 batch_size 만큼의 데이터를 생성하며, 입력은 [이미지 배치, 각도 배치] 형태의 리스트이고, 출력은 정답 레이블 배치입니다!
  
    - 좀 더 구체적인 설명 :
  
        1. `get.flow(X1,y, ...)` 에서 y를 인자로 받는 이유
        - 결론 : 증강된 이미지 X1와 그 짝이 되는 정답 y의 연결고리를 절대 놓치지 않기 위해서.
        - 모델은 (입력, 정답) 쌍을 보고 학습합니다. 즉, '이런 이미지가 들어오면, 정답은 이것이다'라는 관계를 배우는 것.
        - `ImageDataGenerator`는 다음과 같은 일을 합니다.
            1. 원본 이미지 데이터 X1를 가져옵니다.
            2. 무작위로 뒤섞습니다. Shuffle
            3. 무작위로 변형 augmentation을 가합니다
        - 만약 X1만 gen.flow에 넘겨준다면, 제너레이터는 이미지를 뒤섞고 변형한 뒤 새로운 순서의 이미지 배치를 만들어낼 겁니다. 하지만, 우리는 새로운 순서에 맞는 정답 y의 순서를 알 수 없게됩니다. 원래 5번 인덱스에 있던 '빙산'의 이미지가 무작위 회전 후 섞여서 23번 인덱스가 되었는데, 정작 23번 인덱스의 정답은 엉뚱한 '배'가 될 수 있는 것이죠.
        - gen.flow(X1,y)는 이 문제를 완벽하게 해결합니다.
        - flow 메서드에 X1와 y를 함께 넘겨주면, 제너레이터는 내부적으로 X1과 y를 한쌍으로 묶어서 관리합니다. 그래서, X1의 순서를 섞을 때, y의 순서도 정확히 동일하게 섞어줍니다. X1의 5번 이미지를 10도 회전시켜 배치에 포함시킬 때, y의 5번 정답을 함께 가져와 그 이미지의 짝으로 붙여줍니다.
        2. batch_size 설명
        - batch_size란, 모델이 가중치를 한 번 업데이트하기 위해, 학습하는 데이터 샘플의 개수를 의미합니다.
     
        - 딥러닝 모델은 수많은 데이터를 보고 학습하지만, 전체 데이터를 한 번에 보고 학습하는 경우는 거의 없습니다! 대신, 데이터를 작은 묶음(mini-batch)으로 나누어 학습을 진행합니다.
     
            - batch_size = 1 : Stochastic Gradeint Descent : 데이터 1개를 보고, 오차를 계산한 뒤, 즉시 가중치를 업데이트. 학습 과정이 매우 불안정하고 들쭉날쭉합니다.
            - batch_size = 전체 데이터 개수 : Batch Gradient Descent : 전체 데이터를 모두 보고, 모든 오차의 평균을 계산한 뒤, 가중치를 딱 한 번 업데이트합니다. 학습 과정이 매우 안정적이지만, 메모리 요구량이 엄청나고 계산에 시간이 너무 오래 걸립니다.
            - batch_size = 32, 64, 128...(Mini-batch Gradient Descent) : 위 두 방법의 절충안.
                1. 메모리 효율성 : 전체 데이터를 메모리에 올릴 필요 없이, 배치 크기만큼의 데이터만 있으면 되므로 효율적.
                2. 계산 효율성 : GPU는 여러 계산을 병렬로 처리하는 데 특화. 데이터 1개를 64번 처리하는 것보다, 64개를 한 묶음으로 처리하는 것이 훨씬 빠릅니다!
                3. 안정적인 학습. : 1개의 데이터는 노이즈가 심할 수 있지만, 32개 데이터의 평균 오차는 전체 데이터의 경향성을 더욱 안정적으로 대표!
        
        3. Flow 메서드
        - 역할 : 이미 메모리에 불러와 있는 NumPy 배열 형태의 데이터를 입력 받아, 실시간으로 데이터 증강을 적용하고 지정된 batch_size만큼의 데이터를 계속해서 만들어내는 제너레이터 객체를 생성.
        - 작동과정 :
            1. X 이미지 데이터와 y 정답 데이터를 입력으로 받음
            2. 내부적으로 ImageDataGenerator에서 정의된 증강 규칙과 배치 크기를 기억
            3. model.fit()이 제너레이터에게 데이터를 요청하면, flow는 다음 작읍을 수행
                - 전체 데이터 (X, y) 중에서 batch_size만큼의 데이터를 무작위로 샘플링.
                - 샘플링된 이미지X에 증강 규칙을 적용하여 새로운 이미지 형성
                - 증강된 이미지와 그에 해당하는 정답 y를 한 쌍으로 묶어 (배치 이미지, 배치 정답) 형태로 반환 yield합니다.
                - return vs yield
                    - return :  함수 내에서 return을 만나면, 함수는 값을 반환하고 완전히 종료됨. 함수 내의 모든 지역 변수와 상태는 사라짐.
                    - yield : 함수 내에서 yield를 만나면, 함수는 값을 반환하고, 그 자리에서 실행을 일시 정지. 함수 내의 모든 지역 변수와 상태는 그대로 보존. 다음에 이 함수가 다시 호출되면, 멈췄던 그 다음 줄부터 실행을 재개.
            4. model.fit()은 이 데이터를 받아 학습하고, 다시 제너레이터에게 다음 배치를 요청. 이 과정이 steps_per_epoch만큼 반복.
  
3. get_callbacks 함수 : 훈련 중에 특정 조건이 만족되면 호출되는 함수들(콜백)을 설정

    - earlystopping : 검증 손실 val_loss이 patience = 10 에포크 동안 개선되지 않으면 훈련을 자동으로 중단시켜 시간 낭비와 과적합을 방지.
    - ModelCheckPoint : 검증 손실이 이전보다 낮아질 때만, save_best_only = True 모델의 가중치를 파일 filepath에 저장


Q. 왜 yield가 딥러닝에서 필수적인가?
- 메모리 효율성 때문입니다.
수십만 장의 이미지 데이터를 증강한다고 생각해 보세요. 만약 return을 사용한다면, 증강된 이미지 수십만 장을 모두 메모리에 만들어 올린 뒤 반환해야 합니다. 이는 엄청난 메모리 낭비이며, 데이터가 크면 아예 불가능합니다.

- 하지만 yield를 사용하는 제너레이터는 다릅니다.
model.fit()이 데이터 한 배치를 요청하면, 제너레이터는 정확히 한 배치 분량의 데이터만 만들어서 yield로 전달합니다. 모델이 그 배치로 학습하는 동안 제너레이터는 잠시 쉬고 있습니다. 학습이 끝나고 모델이 다음 배치를 요청하면, 제너레이터는 다시 깨어나 다음 한 배치만 만들어서 전달합니다.

- 즉, 아무리 큰 데이터셋이라도 메모리에는 항상 딱 한 배치 크기만큼의 데이터만 올라와 있게 됩니다. while True: 루프와 yield의 조합은 이 과정을 무한히 반복하며, fit 메서드에 마르지 않는 데이터의 샘을 제공하는 것과 같습니다. 이것이 바로 Keras 제너레이터가 효율적으로 동작하는 핵심 원리입니다.

##### 블록 5 다중 입력 VGG16 모델 정의 Defining the Multi-input VGG16 Model

In [9]:
import tensorflow as tf # 혹은 from keras.optimizers import schedules

def getVggAngleModel():
    # 두 번째 입력 : 각도 데이터 (1개의 값을 가짐)
    input_2 = Input(shape=[1], name = 'angle')
    # 각도 입력을 위한 간단한 Dense 레이어
    angle_layer = Dense(1)(input_2)

    # 기본 모델 : VGG 16 (ImageNet 가중치 사용, 상위 분류층은 제외)
    base_model = VGG16(weights = 'imagenet', include_top = False,
                      input_shape = X_train.shape[1:], classes = 1)

    # VGG 16의 block5_pool 레이어의 출력을 가져옴
    x = base_model.get_layer('block5_pool').output

    # 이미지 특징을 1차원 벡터로 변환
    x = GlobalMaxPooling2D()(x)

    # 이미지 특징 벡터와 각도 특징 벡터를 결합 (concatenate)
    merge_one = concatenate([x, angle_layer])

    # 결합된 특징을 기반으로 새로운 분류층을 쌓음
    merge_one = Dense(512, activation = 'relu', name = 'fc2')(merge_one)
    merge_one = Dropout(0.3)(merge_one)
    merge_one = Dense(512, activation = 'relu', name = 'fc3')(merge_one)
    merge_one = Dropout(0.3)(merge_one)

    # 최종 예측 : 1개의 노드와 sigmoid 활성화 함수를 사용한 이진 분류
    predictions = Dense(1, activation = 'sigmoid')(merge_one)

    # 최종 모델 정의 : 입력은 2개 (이미지, 각도), 출력은 predictions
    model = Model(inputs = [base_model.input, input_2], outputs = predictions)

    # 학습률 스케줄러 정의
    # 초기 학습률 0.001에서 시작하여, 10000 스텝마다 0.9배씩 학습률을 감소시킴
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=1e-3,
        decay_steps=10000,
        decay_rate=0.9
    )

    # 스케줄러를 learning_rate 파라미터에 직접 전달
    sgd = SGD(learning_rate=lr_schedule, momentum=0.9, nesterov=True)
    
    # 모델 컴파일 : 손실함수, 옵티 마이저, 평가 지표 설정
    #sgd = SGD(learning_rate=1e-3, momentum = 0.9, nesterov = True)
    model.compile(loss = 'binary_crossentropy',
                 optimizer = sgd,
                 metrics = ['accuracy'])
    return model

#### 상세 설명 :
- 이 함수는 전이학습과 다중 입력을 결합한 강력한 모델을 구축!

모델 설계 철학 : 왜 이 구조를 사용하는가?
이 모델의 핵심 목표는 두 종류의 다른 정보, 즉 위성 이미지(band_1, band_2)와 입사각(inc_angle)을 모두 활용하여 빙산을 탐지하는 것.

- 이미지 정보 : 객체의 형태, 질감 등 시각적 특징을 담고 있음.
- 각도 정보 : 이미지가 어떤 각도에서 촬영되었는 지를 나타내는 수치 데이터로, 이미지 해석에 중요한 추가 단서.

이처럼 성격이 다른 두 데이터를 효과적으로 결합하기 위해, Keras의 Sequential API가 아닌, 더 유연하고 강력한 Functional API를 사용. 이를 통해, 각 데이터를 별도의 경로로 처리한 뒤, 중간에 하나로 합치는 '두 갈래(two-stream)' 구조의 모델 설계를 할 수 있음.

---
코드 라인별 상세 설명

### 1단계 : 입력 정의

```
# 두 번째 입력: 각도 데이터 (1개의 값을 가짐)
input_2 = Input(shape=[1], name='angle') # shape=[1] 1차원의 값을 가짐
# 각도 입력을 위한 간단한 Dense 레이어
angle_layer = Dense(1)(input_2)
```

- `input_2 = Input(shape=[1], name = 'angle')`:
    - `Input` : Funtional API에서 모델의 입력 지점을 선언하는 레이어. (Input Layer) "여기서 데이터가 들어온다"라고 알려주는 문과 같음.
    - `shape=[1]` : 입력될 데이터의 형태를 정의합니다. inc_angle은 단일 숫자 값이므로, `[1]` 즉, 1개의 원소를 가진 벡터 형태임을 명시합니다.
    - Q. 왜 '[1]'로 정의할까? 각도의 데이터는 (샘플수x75x75)가 아닌가?
    - `name='angle'` : 입력의 이름을 'angle'로 지정. 모델 구조를 시각화하거나 나중에 특정 레이어를 참조할 때, 가독성과 편의성을 높여줌.
- `angle_layer = Dense(1)(input_2)`:
    - 이 부분이 매우 중요. input_2를 바로 사용하지 않고, 왜 Dense(1) 레이어를 통과시킬까?
    - 역할 : 이는 각도 데이터에 대한 **학습 가능한 가중치 weight와 편향 bias**를 만들어주는 역할을 함. 즉, 모델이 '이 각도 값에 얼마만큼의 중요도(가중치)'를 부여할지를 스스로 학습하게 됨. 단순히 원시 (raw) 데이터를 사용하는 것보다 모델에 더 큰 유연성을 부여하며, 각도 정보를 더 의미 있는 특징으로 변환하는 과정.
    - Q. 여기서 Dense(1)에서 1이 의미하는 건 무엇인가? 각각의 각도에 가중치를 준다면, (75x75)픽셀 전부에 가중치를 준다는 것인가?

### 2단계 : 특징 추출기 정의 (Defining the Feature Extractor)
```
# 기본 모델 : VGG16 (ImageNet 가중치 사용, 상위 분류층은 제외)
base_model = VGG16(weights = 'imagenet', include_top = False,
                    input_shape=X.train.shape[1:], classes=1)

#VGG16의 'black5_poop' 레이어의 출력을 가져옴
x = base_model.get_layer('block5_pool').output
```

- `base_model = VGG16(,,,)`:
    - `weights = 'imagenet'`: 우리는 VGG16 모델을 밑바닥부터 훈련시키는 것이 아니다! 수백만 장의 다양한 이미지가 담긴 'imagenet' 데이터셋으로 이미 훈련이 사전에 완료된, 사전 학습된 가중치를 그대로 사용하겠다는 의미! 이것이 바로 전이 학습의 핵심!
    - `include_top` = False : imageNet은 1000개의 클래스를 분류하는 문제였다. `top`은 바로 이 1000개를 분류하기 위한 마지막 완전 연결 계층 fully connected Layers를 의미함. 우리는 1000개 분류가 아닌 빙산/배 분류를 할 것이므로, 이 '머리' 부분은 필요 없음.
      `include_top = Flase`는 이 머리 부분을 잘라내고, 이미지의 보편적인 특징(선, 면, 질감, 형태 등)을 추출하는 강력한 '몸통' 부분만 사용하겠다는 의미.
    - input_shape = X_train.shape[1:] : 우리가 사용할 이미지의 크기 (높이, 너비, 채널 수)를 모델에게 알려줍니다. VGG16의 몸통 부분이 이 형태에 맞춰 구성됩니다.
    - Q. 여기에서 X_train.shape[1:]는 어떤 쉐입인가?
 - `x = base_model.get_layer('block5_pool').output` :
     - VGG16 모델의 여러 레이어 중에서, 이름이 `block5_pool`인 레이어를 찾아, 그 출력 텐서를 `x`에 저장. 이는 VGG16의 몸통 부분 중에서도 가장 깊은 곳에서 나오는, 고차원적이고 압축된 특징 정보를 사용하겠다는 의미.

### 3단계 : 특징 벡터화 및 결합 Feature Vectorization & Merging
```
# 이미지 특징을 1차원 벡터로 변환
x = GlobalMaxPooling2D()(x)

# 이미지 특징 벡터와 각도 특징 벡터를 결합 (concatenate)
merge_one = concatenate([x, angle_layer])
```

- `x = GlobalMaxPooling2D()(x)`:
    - `block5_pool`의 출력은 아직 2차원의 공간 정보를 가진 특징 맵 형태 (e.g., 4 x 4 x 512). 이를 분류기에 넣기 위해 1차원 벡터로 만들어야 합니다. GlobalMaxPooling2D는 각 채널 512개에서 가장 중요한 특징 값 하나만을 뽑아, (512, )크기의 1차원 벡터로 변환. 이는 과적합을 줄이면서도 핵심 특징을 효과적으로 요약.

- `merge_one = concatenate([x, angle_layer])` :
    - 이 모델의 심장부. 두 갈래로 들어온 정보가 여기에서 하나로 합쳐집니다!
    - 이미지로부터 추출된 특징 벡터 x(크기 512)와, 각도 정보로부터 변환된 특징 벡터 angler_layer (크기 1)을 이어 붙여, 총 (513,) 크기의 새로운 통합 특징 벡터를 만듭니다. 이제 모델은 이 통합된 정보를 바탕을 최종 판단을 내릴 준비를 합니다.

### 4단계 : 맞춤형 분류기 구축 Building the Custom Classifier
```
# 결합된 특징을 기반으로 새로운 분류층을 쌓음.
merge_one = Dense(512, activation = 'relu', name = 'fc3')(merge_on)
merge_one = Dropout(0.3)(merge_one)
merge_one = Dense(512, activation = 'relu', name = 'fc3')(merge_on)
merge_one = Dropout(0.3)(merge_one)

# 최종 예측 : 1개의 노드와 sigmoid 활성화 함수를 사용한 이진 분류
predictions = Dense(1, activation = 'sigmoid')(merge_one) 
```

- Dense와 Dropout 층들:

    - 이 부분은 include_top=False로 잘라냈던 '머리' 부분을 우리 문제에 맞게 새로 만들어 붙이는 과정입니다.
    - Dense(512, activation='relu'): 513개의 통합 특징을 입력받아 512개의 뉴런으로 더 복잡한 패턴을 학습합니다.
    - Dropout(0.3): 훈련 시 30%의 뉴런을 무작위로 비활성화하여, 모델이 특정 특징에만 과도하게 의존하는 것을 막고 과적합을 방지합니다.
참고: 코드에서 name='fc2'가 두 번 사용되었는데, 두 번째는 name='fc3'로 하는 것이 일반적인 관례입니다.
- `predictions = Dense(1, activation='sigmoid'):`
    - 최종 출력 레이어입니다.
    - Dense(1): '빙산이다 / 아니다' 라는 이진 분류(binary classification) 문제이므로, 최종 출력 뉴런은 1개입니다.
    - activation='sigmoid': Sigmoid 함수는 어떤 입력값이든 0과 1 사이의 값으로 변환해 줍니다. 이 출력값은 '이미지가 빙산일 확률'로 해석될 수 있습니다. (e.g., 0.9는 빙산일 확률이 90%임을 의미)

### 5단계 : 모델 정의 및 컴파일 Model Definition & Compilation
```
# 최종 모델 정의: 입력은 2개(이미지, 각도), 출력은 predictions
model = Model(inputs=[base_model.input, input_2], outputs=predictions)

# 모델 컴파일: 손실 함수, 옵티마이저, 평가 지표 설정
sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='binary_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])
return model
```
- model = Model(...):

    - Functional API를 사용하여 최종 모델의 청사진을 완성합니다. "이 모델은 base_model.input(이미지)과 input_2(각도)라는 두 개의 입력을 받아서, 복잡한 내부 경로를 거친 뒤 predictions라는 하나의 출력을 내보낸다"고 명확히 정의합니다.
- model.compile(...):
    - 만들어진 모델을 훈련할 수 있도록 '설정'하는 과정입니다.
    - loss='binary_crossentropy': 이진 분류 문제에서 확률 기반 예측(sigmoid)의 오차를 측정하는 가장 표준적인 손실 함수입니다.
    - optimizer=sgd: 앞서 설정한 SGD 옵티마이저를 사용하여 손실을 최소화하도록 가중치를 업데이트합니다.
    - metrics=['accuracy']: 훈련 과정에서 손실과 더불어 '정확도'도 함께 기록하고 모니터링합니다.








Q. Functional API이란?

1. Keras의 Functional API
Functional API는 Keras에서 신경망 모델을 만드는 두 가지 방법 중 하나로, Sequential API보다 훨씬 유연하고 강력한 방식입니다.

- Sequential API가 마치 레고 블록을 순서대로 차곡차곡 쌓아 올리는 것처럼 단순한 모델을 만드는 데 적합하다면, Functional API는 레이어를 함수처럼 다루어 복잡한 그래프(Graph) 구조를 만들 수 있게 해줍니다.

- 핵심 아이디어
    - 레이어를 하나의 함수라고 생각하는 것입니다. output_tensor = MyLayer()(input_tensor) 처럼, 입력 텐서(데이터)를 레이어 함수에 통과시켜 출력 텐서를 얻습니다. 이 과정을 자유롭게 연결하여 모델의 데이터 흐름을 직접 설계합니다.

- Functional API가 꼭 필요한 경우 (장점)
    - Sequential API로는 구현이 불가능하거나 매우 어려운, 다음과 같은 복잡한 모델을 만들 수 있습니다.

- 다중 입력 (Multi-input) 및 다중 출력 (Multi-output) 모델:

    - 이번에 분석한 getVggAngleModel처럼 이미지와 각도라는 두 개의 다른 입력을 받는 모델이 대표적인 예입니다.
    - 하나의 입력으로 여러 가지를 동시에 예측하는 모델(e.g., 이미지 한 장으로 객체의 종류와 위치를 모두 예측)도 만들 수 있습니다.
- 레이어 공유 (Shared Layers):

    - 하나의 레이어 인스턴스를 여러 입력에 재사용하여 가중치를 공유하는 모델을 만들 수 있습니다. 예를 들어, 두 문장의 유사도를 비교할 때 동일한 임베딩 레이어를 사용하는 경우가 있습니다.
- 비선형적 토폴로지 (Non-linear Topology):

    - 데이터의 흐름이 중간에 여러 갈래로 나뉘었다가 다시 합쳐지는 Inception 모델이나, 잔차 연결(residual connection)이 있는 ResNet 같은 복잡한 구조를 구현할 수 있습니다.

- 결론적으로, Functional API는 딥러닝 아키텍처를 훨씬 더 자유롭게 설계할 수 있게 해주는 전문가용 도구라고 할 수 있습니다.







```
[입력 1: 이미지]         [입력 2: 각도]
      |                       |
      ▼                       ▼
[VGG16의 '눈']        [Dense(1) 레이어]
(특징 추출기)          (특징 변환기)
      |                       |
      ▼                       ▼
[이미지 특징 벡터]      [각도 특징 벡터]
(e.g., 512개 값)      (e.g., 1개 값)
      |                       |
      +-------(concatenate)-------+
                  |
                  ▼
            [merge_one]
         (통합 특징 벡터, 513개 값)
                  |
                  ▼
        [우리만의 '새로운 두뇌']
       (Dense + Dropout 레이어들)
                  |
                  ▼
          [최종 예측 (확률)]
```

#### Question 1. 모델 구축의 워크플로우는 어떻게 되는가?
<다중 입력 모델 설계 4단계 워크플로우>

1. 1단계 (



1. 복잡한 모델 구축 흐름, 정립하기
복잡해 보이는 Functional API의 모델 구축 과정은 사실 매우 체계적인 4단계의 설계도를 따릅니다. 앞으로 어떤 복잡한 모델을 보시더라도 이 4단계 프레임워크에 대입해 보면 구조가 한눈에 들어올 겁니다.

<다중 입력 모델 설계 4단계 워크플로우>

1단계: 재료 준비 (입력 정의)
목표: 우리 모델이 어떤 종류의 데이터를 받을지 모두 선언합니다.
실행: Input() 레이어를 사용해 모델의 '입구'를 모두 만듭니다. 우리 모델은 이미지와 각도, 두 가지 재료가 필요하니 입구도 두 개를 만듭니다.
image_input = Input(shape=[75, 75, 3])
angle_input = Input(shape=[1])
2단계: 재료별 손질 (개별 특징 추출)
목표: 각 재료의 특성에 맞게 개별적으로 전처리하거나 특징을 추출합니다.
실행: 각 입력에 맞는 '처리 공정'을 따로따로 설계합니다.
이미지 공정: 이미지(image_input)는 VGG16이라는 강력한 '분쇄기'에 넣어 고차원적인 특징 벡터로 압축합니다.
각도 공정: 각도(angle_input)는 Dense(1)이라는 간단한 '변환기'를 통과시켜 모델이 사용하기 좋은 특징으로 만듭니다.
3단계: 재료 혼합 (특징 결합)
목표: 각자 손질된 재료들을 하나의 그릇에 담아 합칩니다.
실행: concatenate 레이어를 사용해 각 공정에서 나온 특징 벡터들을 하나로 이어 붙입니다.
merged_features = concatenate([image_features, angle_features])
이 merged_features가 바로 merge_one입니다. 이제 여기에는 이미지 정보와 각도 정보가 모두 담겨 있습니다.
4단계: 최종 조리 (예측)
목표: 잘 섞인 재료를 오븐에 넣고 구워 최종 결과물을 만듭니다.
실행: 합쳐진 merged_features를 입력으로 받아, 우리 문제에 특화된 완전 연결 신경망(Dense 레이어들)을 통과시켜 최종 예측 확률을 출력합니다.
이 4단계, 정의 → 추출 → 결합 → 예측 흐름을 기억하시면, 앞으로 어떤 복잡한 모델을 보더라도 길을 잃지 않고 그 구조를 분석하실 수 있을 겁니다.

2. Functional API의 () () 작동 방식
이 부분은 파이썬의 객체 지향 프로그래밍과 Keras의 설계 철학이 결합된 결과입니다. 아주 좋은 질문이에요.

결론부터 말하면, 첫 번째 괄호 ()는 '레이어 객체 생성 및 설정'이고, 두 번째 괄호 ()는 '해당 레이어에 입력 데이터를 통과시키는 실행'입니다.

비유: '기계'를 만들고 '작동'시키기
Dense(512, activation='relu')와 같은 코드를 '기계'에 비유해 보겠습니다.

Dense(512, activation='relu'): 이 부분은 기계를 만드는 과정입니다.

Dense라는 종류의 기계를 만들기로 결정하고, 첫 번째 괄호 () 안에 기계의 설정을 지정합니다. "뉴런은 512개짜리로 하고, 활성화 함수는 'relu' 다이얼에 맞춰줘."
이 코드가 실행된 시점에서, 설정이 완료된 Dense 기계 **한 대(객체)**가 메모리에 만들어졌지만, 아직 아무 일도 하지 않고 그냥 서 있는 상태입니다.
(...) (merge_one): 이 두 번째 괄호가 바로 기계를 작동시키는 스위치입니다.

"자, 이제 방금 만든 그 Dense 기계에 merge_one이라는 재료를 넣고 작동시켜!" 라는 의미입니다.
기계는 merge_one을 입력으로 받아 내부 계산을 수행한 뒤, 그 결과물(출력 텐서)을 뱉어냅니다.
이것을 코드로 풀어쓰면 다음과 같습니다.

Python

# 1. 기계를 먼저 만들고 설정합니다.
my_dense_layer = Dense(512, activation='relu', name='fc3')

# 2. 만들어진 기계에 데이터를 넣어 작동시키고, 그 결과를 받습니다.
output = my_dense_layer(merge_one) 
Keras에서는 편의를 위해 이 두 줄의 코드를 한 줄로 합쳐서 output = Dense(512, ...)(merge_one) 처럼 쓸 수 있게 해주는 것입니다. GlobalMaxPooling2D()(x) 도 마찬가지 원리입니다. 'GlobalMaxPooling2D 기계'를 만들자마자, 바로 x를 넣어 작동시키는 것이죠.

"원래 존재하는 merge_one에 add.Dense() 하는 것이라고 이해해도 되는거야?"

이 부분이 정말 중요한 포인트입니다. 그렇게 이해하시면 안 됩니다. Functional API는 데이터의 흐름을 정의하는 방식입니다. Dense(...)(merge_one)은 merge_one이라는 텐서를 수정하는 것이 아니라, merge_one이 Dense 레이어를 통과하여 나온 새로운 출력 텐서를 만드는 것입니다. 우리는 이 새로운 출력 텐서를 다음 레이어의 입력으로 계속 연결해 나가는 것입니다.

3. shape=[1]과 배치(Batch)의 비밀
이것도 정말 많은 분들이 헷갈려 하는 부분입니다. 명확하게 정리해 드릴게요.

핵심 규칙: Input 레이어를 정의할 때는, 데이터 '한 개'의 모양(shape)만 알려주면 됩니다. Keras는 배치(batch) 차원을 자동으로 처리해 줍니다.

shape=[1]의 진짜 의미
inc_angle 데이터 하나는 39.24 와 같은 단일 숫자입니다. 이 숫자 하나를 벡터로 표현하면 [39.24] 가 되고, 이 벡터의 모양은 (1,) 입니다. 즉, 1개의 원소를 가진 벡터라는 의미죠.

shape=[1]은 바로 "이 입구(Input)로는 데이터 샘플 하나가 들어올 때 (1,) 모양을 가질 거야" 라고 모델에게 알려주는 것입니다. 만약 75x75 RGB 이미지라면 shape=[75, 75, 3] 이라고 알려주는 것과 같습니다.

"나는 배치를 통째로 넣는 줄 알았는데."
네, 실제로 모델을 훈련시킬 때는 batch_size만큼의 데이터를 통째로 넣는 것이 맞습니다. 여기서 마법이 일어납니다.

우리가 shape=[1]로 입력을 정의하면, Keras는 내부적으로 이 입력의 전체 모양을 (None, 1) 로 간주합니다.

None: 이 부분이 바로 **배치 크기를 위한 자리(placeholder)**입니다. None은 "여기는 어떤 숫자든 올 수 있다"는 의미입니다. 훈련 시 batch_size가 32이면 (32, 1) 크기의 데이터가 들어오고, 64이면 (64, 1) 크기의 데이터가 들어오는 것을 알아서 처리해 줍니다.
1: 이 부분이 우리가 정의한 데이터 샘플 하나의 모양입니다.
비유: Input 레이어는 **'USB 포트'**를 설계하는 것과 같습니다. 우리는 'USB 메모리 스틱 한 개'가 들어갈 수 있도록 포트의 규격(shape=[1])을 설계합니다. 이 포트를 설계할 때 "32개짜리 USB 상자가 들어올 거야"라고 가정하지 않습니다. 하지만 일단 포트가 만들어지면, 32개짜리 상자에서 스틱을 가져오든 64개짜리 상자에서 가져오든 문제없이 데이터를 처리할 수 있습니다. None이 바로 이 유연성을 부여하는 것입니다.

마지막 질문에 대한 답변
"뉴럴 네트워크에서는 Input도, 하나의 레이어로 보는거야?"

네, 맞습니다. Input은 실제 계산을 하지는 않지만, 모델의 입력을 정의하고 데이터의 shape과 dtype을 알려주는 역할을 하는 특별한 종류의 레이어입니다. Functional API 모델은 반드시 이 Input 레이어로 시작합니다.
"Dense(1)(input_2)를 통해, 인풋 레이어 다음에 새로운 레이어를 붙여주는거고?"

네, 정확합니다! 데이터의 흐름을 정의하는 것입니다. "데이터는 input_2라는 입구를 통해 들어와서, 곧바로 Dense(1)이라는 첫 번째 처리 공정으로 흘러 들어간다"는 경로를 설계한 것입니다.

In [10]:
def myAngleCV(X_train, X_angle, X_test, target_train): # target_train을 인자로 받도록 수정
    K = 3
    # StratifiedKFold를 사용하여 데이터를 3개의 fold로 나눔
    folds = list(StratifiedKFold(n_splits=K, shuffle=True, random_state=16).split(X_train, target_train))
    
    # 예측 결과를 저장할 배열 초기화
    y_test_pred_log = 0
    y_train_pred_log = 0
    y_valid_pred_log = np.zeros_like(target_train, dtype='float32') # [수정] 더 명확한 배열 초기화

    # 각 fold에 대해 훈련과 검증을 반복
    for j, (train_idx, test_idx) in enumerate(folds):
        print(f'\n=================== FOLD {j+1}/{K} ===================') # [개선] f-string 사용
        # 현재 fold에 맞는 훈련/검증 데이터 분리
        X_train_cv, y_train_cv = X_train[train_idx], target_train[train_idx]
        X_holdout, Y_holdout = X_train[test_idx], target_train[test_idx]
        
        # Angle 데이터도 동일하게 분리
        X_angle_cv, X_angle_hold = X_angle[train_idx], X_angle[test_idx]

        # 콜백을 위한 파일 경로 정의
        file_path = f"{j}_aug_model_weights.keras" # [개선] f-string 및 .keras 확장자 사용 권장
        callbacks = get_callbacks(filepath=file_path, patience=5)
        
        # 데이터 증강 제너레이터 생성
        gen_flow = gen_flow_for_two_inputs(X_train_cv, X_angle_cv, y_train_cv)
        
        # 매번 새로운 모델 생성
        galaxyModel = getVggAngleModel()
        
        # [가장 중요한 수정] fit_generator -> fit 으로 변경
        galaxyModel.fit(
            gen_flow,
            steps_per_epoch=24,
            epochs=100,
            verbose=1,
            validation_data=([X_holdout, X_angle_hold], Y_holdout),
            callbacks=callbacks
            # [수정] shuffle=True 인자 제거
        )

        # 훈련 후 최고 성능의 가중치를 다시 불러옴
        galaxyModel.load_weights(filepath=file_path)
        
        # 훈련 및 검증 세트에 대한 성능 평가
        score_train = galaxyModel.evaluate([X_train_cv, X_angle_cv], y_train_cv, verbose=0)
        print(f'Fold {j+1} Train loss: {score_train[0]:.4f}, Train accuracy: {score_train[1]:.4f}')
        score_valid = galaxyModel.evaluate([X_holdout, X_angle_hold], Y_holdout, verbose=0)
        print(f'Fold {j+1} Test/Holdout loss: {score_valid[0]:.4f}, Test/Holdout accuracy: {score_valid[1]:.4f}')

        # 검증 세트에 대한 예측 수행 (Out-of-Fold 예측)
        pred_valid = galaxyModel.predict([X_holdout, X_angle_hold])
        y_valid_pred_log[test_idx] = pred_valid.flatten() # [개선] .reshape 대신 .flatten() 사용

        # 전체 테스트 세트에 대한 예측 수행 및 누적 (앙상블)
        temp_test = galaxyModel.predict([X_test, X_test_angle])
        y_test_pred_log += temp_test.flatten()

        # (선택적) 전체 훈련 세트에 대한 예측 수행 및 누적
        temp_train = galaxyModel.predict([X_train, X_angle])
        y_train_pred_log += temp_train.flatten()

    # K개의 모델이 예측한 결과의 평균을 최종 예측값으로 사용
    y_test_pred_log = y_test_pred_log / K
    y_train_pred_log = y_train_pred_log / K

    print('\n--- Final Cross-Validation Scores ---')
    print(f'Overall Train Log Loss (In-Fold): {log_loss(target_train, y_train_pred_log):.4f}')
    print(f'Overall Valid Log Loss (Out-of-Fold): {log_loss(target_train, y_valid_pred_log):.4f}')
    
    return y_test_pred_log

In [11]:
preds=myAngleCV(X_train, X_angle, X_test, target_train)


=================== FOLD 1/3 ===================


I0000 00:00:1749709380.441093      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1749709380.441747      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/100


I0000 00:00:1749709387.154005      65 service.cc:148] XLA service 0x7849bda0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1749709387.154851      65 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1749709387.154873      65 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1749709387.584114      65 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1749709399.188330      65 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


24/24 ━━━━━━━━━━━━━━━━━━━━ 37s 859ms/step - accuracy: 0.6019 - loss: 0.7866 - val_accuracy: 0.6430 - val_loss: 0.5559
Epoch 2/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 5s 221ms/step - accuracy: 0.7334 - loss: 0.4811 - val_accuracy: 0.8505 - val_loss: 0.3165
Epoch 3/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 5s 195ms/step - accuracy: 0.8144 - loss: 0.3708 - val_accuracy: 0.8766 - val_loss: 0.2896
Epoch 4/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 5s 201ms/step - accuracy: 0.8877 - loss: 0.2689 - val_accuracy: 0.8710 - val_loss: 0.2823
Epoch 5/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 5s 191ms/step - accuracy: 0.8827 - loss: 0.2579 - val_accuracy: 0.8953 - val_loss: 0.2532
Epoch 6/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 4s 170ms/step - accuracy: 0.8964 - loss: 0.2678 - val_accuracy: 0.8841 - val_loss: 0.2576
Epoch 7/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 5s 213ms/step - accuracy: 0.8930 - loss: 0.2446 - val_accuracy: 0.8935 - val_loss: 0.2402
Epoch 8/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 5s 192ms/step - accuracy: 0.8952 - loss: 0.2423 - val_accuracy: 0.912

In [12]:
#Submission for each day.
submission = pd.DataFrame()
submission['id']=test['id']
submission['is_iceberg']=preds
submission.to_csv('sub.csv', index=False)